# 2. Extração de eventos de tramitação no Congresso


Neste notebook, vamos extrair do banco de dados os eventos de tramitação, isto é, etapas que sinalizem o avanço de uma proposição no processo legislativo, p. ex. quando uma proposição é recebida por comissão ou votada em plenário.


## 2.1. Preparação


### 2.1.1. Imports

In [1]:
import json
import os
import re

from pathlib import Path

import duckdb
import pandas as pd

from tramita.gold import Event  # Enum customizado nosso para eventos relevantes de tramitação

from dotenv import load_dotenv

load_dotenv()

PROJECT_DIR = Path("~/tramita").expanduser()
DB_PATH = PROJECT_DIR / os.getenv("SILVER_DUCKDB_PATH", "")
OUT_DIR = PROJECT_DIR / "gold" / "outputs"
OUT_DIR.mkdir(exist_ok=True)
ACCESS_DIR = PROJECT_DIR / "gold" / "accessory_data"
ACCESS_DIR.mkdir(exist_ok=True)

PARQUET_ROOT = PROJECT_DIR / "data" / "bronze" / "snapshots" / "bronze-2020-2024-v2"

NODES_PATH_CSV = OUT_DIR / "nodes.csv"
EDGES_PATH_CSV = OUT_DIR / "edges.csv"

### 2.1.2. Leitura do banco de dados

Aqui carregamos os dados da fase prata. Em todas as tabelas exceto uma, adotamos o ID natural como índice, para que essas tabelas possam ser alvo de *joins*. Apenas para a tabela de tramitações isso não é necessário, já que ela será a base para a construção da tabela final de eventos e seu índice será desprezado.

In [2]:
with duckdb.connect(DB_PATH, read_only=True) as con:
    # Câmara
    ## Proposições
    house_props_df = con.execute("SELECT * FROM proposicoes_camara").df().set_index('id_proposicao', drop=True)
    ## Órgãos
    house_orgaos_df = con.execute("SELECT * FROM orgaos_camara").df().set_index('id_orgao', drop=True)
    ## Tramitações
    house_tram_df = con.execute("SELECT * FROM tramitacoes_camara").df()
    
    # Senado
    ## Processos
    senate_procs_df = con.execute("SELECT * FROM processo_senado").df().set_index('id_processo', drop=True)
    
    ## Despachos
    senate_desp_df = con.execute("SELECT * FROM despachos_senado").df().set_index('id_despacho', drop=True)
    
    ## Providências
    senate_prov_df = con.execute("SELECT * FROM providencias_senado").df().set_index('id_providencia', drop=True)
    
    ## Informes legislativos
    senate_inf_df = con.execute("SELECT * FROM informes_legislativos_senado").df().set_index('id_informe_legislativo', drop=True)
    
    ## Unidades destinatárias
    senate_unid_df = con.execute("SELECT * FROM unidades_destinatarias_senado").df().set_index('id_unidade_destinataria', drop=True)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### 2.1.3. Seleção dos dados relevantes à nossa análise

Primeiro recuperamos os IDs relevantes de proposições e processos do grafo que construímos na parte 1.

In [3]:
nodes_df = pd.read_csv(NODES_PATH_CSV)
prop_ids = nodes_df[nodes_df['type'].eq('Proposicao')]['tag'].str.split(":", expand=True)[1].astype(int).to_list()
proc_ids = nodes_df[nodes_df['type'].eq('Processo')]['tag'].str.split(":", expand=True)[1].astype(int).to_list()

Então selecionamos os dados de tramitação pertinentes somente a essas IDs.

In [4]:

house_tram_df = house_tram_df[
    house_tram_df['id_proposicao'].isin(prop_ids)
].copy().set_index('id_tramitacao', drop=True)

def filter_by_processo(df):
    return df[df['id_processo'].isin(proc_ids)]

senate_desp_df = filter_by_processo(senate_desp_df)
senate_prov_df = filter_by_processo(senate_prov_df)
senate_inf_df = filter_by_processo(senate_inf_df)
senate_unid_df = filter_by_processo(senate_unid_df)

Cabe observar que não vamos filtrar os órgãos da Câmara, uma vez que podem figurar também como locais onde tramitações ocorrem (ex. Comissões, Plenário).

## 2.2. Construção da tabela de tramitações

A partir daqui vamos executar uma série de passos para construir uma tabela completa com as tramitações de cada proposição em cada casa.

### 2.2.1. Câmara dos Deputados

Primeiro fazemos um join com a tabela de vértices para recuperar os nomes das proposições.

In [5]:
house_tram_df['id_proposicao'].isna().any()

np.False_

Derivamos a tabela de *tags* para compatibilizar com nossos vértices.

In [6]:
house_tram_df['prop_tag'] = "CP:" + house_tram_df['id_proposicao'].astype(str)
house_tram_df

,id_proposicao,ambito,apreciacao,cod_situacao,cod_tipo_tramitacao,data_hora,descricao_situacao,descricao_tramitacao,despacho,regime,sequencia,sigla_orgao,uri_orgao,uri_ultimo_relator,year_snapshot,prop_tag
id_tramitacao,,,,,,,,,,,,,,,,
462,538196,Regimental,Proposição Sujeita à Apreciação do Plenário,1140,100,2021-11-17 00:00:00,Transformado em Norma Jurídica,Apresentação de Proposição,"Ofício nº 205/2021/PS-GSE ao Senado Federal, i...","Urgência (Art. 155, RICD)",213,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2020,CP:538196
545,559138,Regimental,Proposição Sujeita à Apreciação do Plenário,1140,100,2025-05-05 00:00:00,Transformado em Norma Jurídica,Apresentação de Proposição,"Apresentação do Of. nº 10 /2024/PS-GSE, que co...","Urgência (Art. 155, RICD)",262,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2020,CP:559138
647,593065,Regimental,Proposição Sujeita à Apreciação do Plenário,1140,100,2021-11-17 00:00:00,Transformado em Norma Jurídica,Apresentação de Proposição,"Ofício nº 201/2021/PS-GSE ao Senado Federal, ...","Urgência (Art. 155, RICD)",322,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2020,CP:593065
687,601739,Regimental,Proposição Sujeita à Apreciação do Plenário,924,501,2024-12-02 18:35:00,Pronta para Pauta,Recebimento - Relator(a),Apresentação do PRL n. 1 CAPADR (Parecer do Re...,"Ordinário (Art. 151, III, RICD)",136,CAPADR,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2020,CP:601739
744,614512,Regimental,Proposição Sujeita à Apreciação do Plenário,1140,500,2022-07-15 00:00:00,Transformado em Norma Jurídica,Recebimento,Recebimento do Ofício nº 306/2022 (CN) encamin...,"Urgência (Art. 155, RICD)",203,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2020,CP:614512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880670,2466104,Regimental,Proposição Sujeita à Apreciação do Plenário,1140,194,2024-11-04 14:35:00,Transformado em Norma Jurídica,Apresentação de Requerimento,Apresentação do REQ n. 4403/2024 (Requerimento...,"Urgência (Art. 155, RICD)",13,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2024,CP:2466104
880671,2466104,Regimental,Proposição Sujeita à Apreciação do Plenário,1140,110,2024-11-04 13:13:00,Transformado em Norma Jurídica,Distribuição,Às Comissões de Administração e Serviço Públic...,"Urgência (Art. 155, RICD)",8,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2024,CP:2466104
880672,2466104,Regimental,Proposição Sujeita à Apreciação do Plenário,None,604,2024-11-04 00:00:00,None,Publicação de Proposição,Encaminhada à publicação. Publicação Inicial e...,"Urgência (Art. 155, RICD)",14,CCP,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/depu...,2024,CP:2466104


Fazemos um *join* com a tabela de vértices para recuperar os nomes das proposições (p. ex. PL 2597/2024).

In [7]:
house_tram_expanded_df = house_tram_df.join(
    nodes_df.set_index('tag', drop=True)['label'],
    on="prop_tag",
    how="left",
)[[
    'id_proposicao',
    'label',
    'sequencia',
    'data_hora',
    'descricao_tramitacao',
    'sigla_orgao',
    'despacho',
    'cod_situacao',
    'descricao_situacao',
    'regime',
    'apreciacao',
    'cod_tipo_tramitacao',
    'uri_orgao',
]].sort_values(['id_proposicao', 'data_hora']).rename(columns={'label': 'prop_label'})
house_tram_expanded_df

,id_proposicao,prop_label,sequencia,data_hora,descricao_tramitacao,sigla_orgao,despacho,cod_situacao,descricao_situacao,regime,apreciacao,cod_tipo_tramitacao,uri_orgao
id_tramitacao,,,,,,,,,,,,,
771001,253500,PL 2597/2024,1,2004-05-13 14:41:00,Apresentação de Proposição,PLEN,Apresentação do Projeto de Lei pelo Dep. José ...,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,100,https://dadosabertos.camara.leg.br/api/v2/orga...
771000,253500,PL 2597/2024,6,2004-05-14 10:46:00,Recebimento,SECAP(SGM),Recebimento pela MESA.,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,500,https://dadosabertos.camara.leg.br/api/v2/orga...
770999,253500,PL 2597/2024,12,2004-05-28 11:08:00,Distribuição,MESA,"Às Comissões de Desenvolvimento Econômico, Ind...",1140,Transformado em Norma Jurídica,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,110,https://dadosabertos.camara.leg.br/api/v2/orga...
770998,253500,PL 2597/2024,17,2004-06-01 00:00:00,Publicação de Proposição,CCP,Encaminhada à publicação. Publicação Inicial n...,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,604,https://dadosabertos.camara.leg.br/api/v2/orga...
770997,253500,PL 2597/2024,18,2004-06-02 09:45:00,Recebimento,CDE,Recebimento pela CDEIC.,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,500,https://dadosabertos.camara.leg.br/api/v2/orga...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
875892,2531284,PL 3194/2025,2,2025-07-02 14:10:00,Apresentação de Proposição,MESA,Apresentação do PL n. 3194/2025 (Projeto de Le...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,100,https://dadosabertos.camara.leg.br/api/v2/orga...
875352,2531284,PL 3194/2025,8,2025-08-29 14:45:00,Despacho de Apensação,MESA,Apense-se à(ao) PL-4348/2024.Proposição Sujeit...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,129,https://dadosabertos.camara.leg.br/api/v2/orga...
874734,2531284,PL 3194/2025,14,2025-09-02 00:00:00,Publicação de Proposição,CCP,Encaminhada à publicação. Publicação Inicial e...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,604,https://dadosabertos.camara.leg.br/api/v2/orga...


Juntamos a isso os nomes e tipos dos órgãos, fazendo um *join* com a tabela de órgãos da Câmara.

In [8]:
house_tram_expanded_df = house_tram_expanded_df.join(
    house_orgaos_df.set_index('uri')[['cod_tipo_orgao', 'nome']],
    on="uri_orgao",
    how="left",
).rename(columns={'nome': 'nome_orgao'}).copy()
house_tram_expanded_df['data_hora'] = pd.to_datetime(house_tram_expanded_df['data_hora'])
house_tram_expanded_df

,id_proposicao,prop_label,sequencia,data_hora,descricao_tramitacao,sigla_orgao,despacho,cod_situacao,descricao_situacao,regime,apreciacao,cod_tipo_tramitacao,uri_orgao,cod_tipo_orgao,nome_orgao
id_tramitacao,,,,,,,,,,,,,,,
771001,253500,PL 2597/2024,1,2004-05-13 14:41:00,Apresentação de Proposição,PLEN,Apresentação do Projeto de Lei pelo Dep. José ...,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,100,https://dadosabertos.camara.leg.br/api/v2/orga...,26,Plenário
771000,253500,PL 2597/2024,6,2004-05-14 10:46:00,Recebimento,SECAP(SGM),Recebimento pela MESA.,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,500,https://dadosabertos.camara.leg.br/api/v2/orga...,12000,Seção de Registro e Controle de Análise da Pro...
770999,253500,PL 2597/2024,12,2004-05-28 11:08:00,Distribuição,MESA,"Às Comissões de Desenvolvimento Econômico, Ind...",1140,Transformado em Norma Jurídica,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,110,https://dadosabertos.camara.leg.br/api/v2/orga...,1,Mesa Diretora da Câmara dos Deputados
770998,253500,PL 2597/2024,17,2004-06-01 00:00:00,Publicação de Proposição,CCP,Encaminhada à publicação. Publicação Inicial n...,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,604,https://dadosabertos.camara.leg.br/api/v2/orga...,12000,COORDENAÇÃO DE COMISSÕES PERMANENTES
770997,253500,PL 2597/2024,18,2004-06-02 09:45:00,Recebimento,CDE,Recebimento pela CDEIC.,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,500,https://dadosabertos.camara.leg.br/api/v2/orga...,2,Comissão de Desenvolvimento Econômico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875892,2531284,PL 3194/2025,2,2025-07-02 14:10:00,Apresentação de Proposição,MESA,Apresentação do PL n. 3194/2025 (Projeto de Le...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,100,https://dadosabertos.camara.leg.br/api/v2/orga...,1,Mesa Diretora da Câmara dos Deputados
875352,2531284,PL 3194/2025,8,2025-08-29 14:45:00,Despacho de Apensação,MESA,Apense-se à(ao) PL-4348/2024.Proposição Sujeit...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,129,https://dadosabertos.camara.leg.br/api/v2/orga...,1,Mesa Diretora da Câmara dos Deputados
874734,2531284,PL 3194/2025,14,2025-09-02 00:00:00,Publicação de Proposição,CCP,Encaminhada à publicação. Publicação Inicial e...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,604,https://dadosabertos.camara.leg.br/api/v2/orga...,12000,COORDENAÇÃO DE COMISSÕES PERMANENTES


Precisamos também dos nomes dos tipos de órgãos. Isso temos da fase bronze.

In [9]:
payloads_str = pd.read_parquet(
    PARQUET_ROOT / "camara" / "referencias" / "orgaos_codTipoOrgao",
)['payload_json'].to_list()
payloads = list(map(json.loads, payloads_str))

house_tipo_orgao_df = (
    pd.DataFrame(payloads)[['cod', 'nome']]
        .rename(columns={'cod': 'cod_tipo_orgao', 'nome': 'tipo_orgao'})
)
house_tipo_orgao_df['cod_tipo_orgao'] = house_tipo_orgao_df['cod_tipo_orgao'].astype(int)
house_tipo_orgao_df['tipo_orgao'] = house_tipo_orgao_df['tipo_orgao'].astype(str)
house_tipo_orgao_df = house_tipo_orgao_df.set_index('cod_tipo_orgao', drop=True)
house_tipo_orgao_df

,tipo_orgao
cod_tipo_orgao,
1,Comissão Diretora
2,Comissão Permanente
3,Comissão Especial
4,Comissão Parlamentar de Inquérito
5,Comissão Externa
6,Comissão Mista Permanente
7,Comissão de Sindicância
8,Comissão Representativa do CN
9,Comissão Medida Provisória


E juntamos isso também à tabela de tramitações.

In [10]:
house_tram_expanded_df = house_tram_expanded_df.join(
    house_tipo_orgao_df,
    on="cod_tipo_orgao",
)
house_tram_expanded_df

,id_proposicao,prop_label,sequencia,data_hora,descricao_tramitacao,sigla_orgao,despacho,cod_situacao,descricao_situacao,regime,apreciacao,cod_tipo_tramitacao,uri_orgao,cod_tipo_orgao,nome_orgao,tipo_orgao
id_tramitacao,,,,,,,,,,,,,,,,
771001,253500,PL 2597/2024,1,2004-05-13 14:41:00,Apresentação de Proposição,PLEN,Apresentação do Projeto de Lei pelo Dep. José ...,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,100,https://dadosabertos.camara.leg.br/api/v2/orga...,26,Plenário,Plenário Virtual
771000,253500,PL 2597/2024,6,2004-05-14 10:46:00,Recebimento,SECAP(SGM),Recebimento pela MESA.,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,500,https://dadosabertos.camara.leg.br/api/v2/orga...,12000,Seção de Registro e Controle de Análise da Pro...,Órgão da Câmara dos Deputados
770999,253500,PL 2597/2024,12,2004-05-28 11:08:00,Distribuição,MESA,"Às Comissões de Desenvolvimento Econômico, Ind...",1140,Transformado em Norma Jurídica,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,110,https://dadosabertos.camara.leg.br/api/v2/orga...,1,Mesa Diretora da Câmara dos Deputados,Comissão Diretora
770998,253500,PL 2597/2024,17,2004-06-01 00:00:00,Publicação de Proposição,CCP,Encaminhada à publicação. Publicação Inicial n...,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,604,https://dadosabertos.camara.leg.br/api/v2/orga...,12000,COORDENAÇÃO DE COMISSÕES PERMANENTES,Órgão da Câmara dos Deputados
770997,253500,PL 2597/2024,18,2004-06-02 09:45:00,Recebimento,CDE,Recebimento pela CDEIC.,None,None,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,500,https://dadosabertos.camara.leg.br/api/v2/orga...,2,Comissão de Desenvolvimento Econômico,Comissão Permanente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875892,2531284,PL 3194/2025,2,2025-07-02 14:10:00,Apresentação de Proposição,MESA,Apresentação do PL n. 3194/2025 (Projeto de Le...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,100,https://dadosabertos.camara.leg.br/api/v2/orga...,1,Mesa Diretora da Câmara dos Deputados,Comissão Diretora
875352,2531284,PL 3194/2025,8,2025-08-29 14:45:00,Despacho de Apensação,MESA,Apense-se à(ao) PL-4348/2024.Proposição Sujeit...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,129,https://dadosabertos.camara.leg.br/api/v2/orga...,1,Mesa Diretora da Câmara dos Deputados,Comissão Diretora
874734,2531284,PL 3194/2025,14,2025-09-02 00:00:00,Publicação de Proposição,CCP,Encaminhada à publicação. Publicação Inicial e...,None,None,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,604,https://dadosabertos.camara.leg.br/api/v2/orga...,12000,COORDENAÇÃO DE COMISSÕES PERMANENTES,Órgão da Câmara dos Deputados


Reorganizamos as colunas:

In [11]:
house_tram_expanded_df = house_tram_expanded_df[[
    # Dados da tramitação
    'sequencia',
    'data_hora',
    'cod_tipo_tramitacao',
    'descricao_tramitacao',
    # Dados da proposição
    'id_proposicao',
    'prop_label',
    # Situação
    'cod_situacao',
    'descricao_situacao',
    # Órgao
    'sigla_orgao',
    'nome_orgao',
    'cod_tipo_orgao',
    'tipo_orgao',
    'uri_orgao',
    # Outros
    'despacho',
    'regime',
    'apreciacao',
]]
house_tram_expanded_df

,sequencia,data_hora,cod_tipo_tramitacao,descricao_tramitacao,id_proposicao,prop_label,cod_situacao,descricao_situacao,sigla_orgao,nome_orgao,cod_tipo_orgao,tipo_orgao,uri_orgao,despacho,regime,apreciacao
id_tramitacao,,,,,,,,,,,,,,,,
771001,1,2004-05-13 14:41:00,100,Apresentação de Proposição,253500,PL 2597/2024,None,None,PLEN,Plenário,26,Plenário Virtual,https://dadosabertos.camara.leg.br/api/v2/orga...,Apresentação do Projeto de Lei pelo Dep. José ...,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário
771000,6,2004-05-14 10:46:00,500,Recebimento,253500,PL 2597/2024,None,None,SECAP(SGM),Seção de Registro e Controle de Análise da Pro...,12000,Órgão da Câmara dos Deputados,https://dadosabertos.camara.leg.br/api/v2/orga...,Recebimento pela MESA.,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário
770999,12,2004-05-28 11:08:00,110,Distribuição,253500,PL 2597/2024,1140,Transformado em Norma Jurídica,MESA,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora,https://dadosabertos.camara.leg.br/api/v2/orga...,"Às Comissões de Desenvolvimento Econômico, Ind...","Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário
770998,17,2004-06-01 00:00:00,604,Publicação de Proposição,253500,PL 2597/2024,None,None,CCP,COORDENAÇÃO DE COMISSÕES PERMANENTES,12000,Órgão da Câmara dos Deputados,https://dadosabertos.camara.leg.br/api/v2/orga...,Encaminhada à publicação. Publicação Inicial n...,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário
770997,18,2004-06-02 09:45:00,500,Recebimento,253500,PL 2597/2024,None,None,CDE,Comissão de Desenvolvimento Econômico,2,Comissão Permanente,https://dadosabertos.camara.leg.br/api/v2/orga...,Recebimento pela CDEIC.,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875892,2,2025-07-02 14:10:00,100,Apresentação de Proposição,2531284,PL 3194/2025,None,None,MESA,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora,https://dadosabertos.camara.leg.br/api/v2/orga...,Apresentação do PL n. 3194/2025 (Projeto de Le...,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...
875352,8,2025-08-29 14:45:00,129,Despacho de Apensação,2531284,PL 3194/2025,None,None,MESA,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora,https://dadosabertos.camara.leg.br/api/v2/orga...,Apense-se à(ao) PL-4348/2024.Proposição Sujeit...,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...
874734,14,2025-09-02 00:00:00,604,Publicação de Proposição,2531284,PL 3194/2025,None,None,CCP,COORDENAÇÃO DE COMISSÕES PERMANENTES,12000,Órgão da Câmara dos Deputados,https://dadosabertos.camara.leg.br/api/v2/orga...,Encaminhada à publicação. Publicação Inicial e...,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...


Por análise manual, identificamos alguns códigos de órgãos que não são relevantes para nosso estudo ou que podem ser erros de digitação.

In [12]:
orgao_code_excludes = [
    70_000,  # Sociedade Civil (provavelmente erro)
    12_000,  # Órgãos burocráticos da Câmara (só tarefas protocolares)
    40_000,  # Senado Federal; veremos lá e não cá
    10,      # Grupos de trabalho, nada relevante
    11,      # Conselho de Ética da Câmara; irrelevante para o escopo deste trabalho
]


Então excluímo-nos da tabela.

In [13]:

house_tram_expanded_filtered_df = house_tram_expanded_df[
    (~house_tram_expanded_df['cod_tipo_orgao'].isin(orgao_code_excludes))
].sort_values(['id_proposicao', 'sequencia'])
house_tram_expanded_filtered_df

,sequencia,data_hora,cod_tipo_tramitacao,descricao_tramitacao,id_proposicao,prop_label,cod_situacao,descricao_situacao,sigla_orgao,nome_orgao,cod_tipo_orgao,tipo_orgao,uri_orgao,despacho,regime,apreciacao
id_tramitacao,,,,,,,,,,,,,,,,
771001,1,2004-05-13 14:41:00,100,Apresentação de Proposição,253500,PL 2597/2024,None,None,PLEN,Plenário,26,Plenário Virtual,https://dadosabertos.camara.leg.br/api/v2/orga...,Apresentação do Projeto de Lei pelo Dep. José ...,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário
770999,12,2004-05-28 11:08:00,110,Distribuição,253500,PL 2597/2024,1140,Transformado em Norma Jurídica,MESA,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora,https://dadosabertos.camara.leg.br/api/v2/orga...,"Às Comissões de Desenvolvimento Econômico, Ind...","Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário
770997,18,2004-06-02 09:45:00,500,Recebimento,253500,PL 2597/2024,None,None,CDE,Comissão de Desenvolvimento Econômico,2,Comissão Permanente,https://dadosabertos.camara.leg.br/api/v2/orga...,Recebimento pela CDEIC.,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário
770996,19,2004-06-08 11:34:00,320,Designação de Relator(a),253500,PL 2597/2024,None,None,CDE,Comissão de Desenvolvimento Econômico,2,Comissão Permanente,https://dadosabertos.camara.leg.br/api/v2/orga...,"Designado Relator, Dep. Ronaldo Dimas (PSDB-TO)","Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário
770995,20,2004-06-08 11:37:00,350,Abertura de Prazo,253500,PL 2597/2024,None,None,CDE,Comissão de Desenvolvimento Econômico,2,Comissão Permanente,https://dadosabertos.camara.leg.br/api/v2/orga...,Abertura de Prazo para Emendas ao Projeto a pa...,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876366,1,2024-04-15 00:00:00,1050,Notificacao para Publicação Intermediária,2531284,PL 3194/2025,925,Tramitando em Conjunto,CDC,Comissão de Defesa do Consumidor,2,Comissão Permanente,https://dadosabertos.camara.leg.br/api/v2/orga...,"Designado Relator, Dep. Márcio Marinho (REPUBL...","Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...
875892,2,2025-07-02 14:10:00,100,Apresentação de Proposição,2531284,PL 3194/2025,None,None,MESA,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora,https://dadosabertos.camara.leg.br/api/v2/orga...,Apresentação do PL n. 3194/2025 (Projeto de Le...,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...
875352,8,2025-08-29 14:45:00,129,Despacho de Apensação,2531284,PL 3194/2025,None,None,MESA,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora,https://dadosabertos.camara.leg.br/api/v2/orga...,Apense-se à(ao) PL-4348/2024.Proposição Sujeit...,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...


A função abaixo extrai um `Event`, onde possível, para cada fileira da tabela de tramitações da Câmara

In [14]:
def house_row_to_event(row: pd.Series) -> Event | None:
    # apresentado
    if row['cod_tipo_tramitacao'] == '100':
        return Event.APRESENTADO

    # distribuído
    # Retirei porque não faz diferença para isso e ser recebido na comissão
    # if row['cod_tipo_tramitacao'] == '110':
    #     return Event.DISTRIBUIDO

    # recebido em comissão
    if row['cod_tipo_tramitacao'] == '500' and row['tipo_orgao'].startswith('Comissão'):
        return Event.RECEBIDO_COMISSAO

    # designado relator em comissão
    if row['cod_tipo_tramitacao'] == '320' and row['tipo_orgao'].startswith('Comissão'):
        return Event.DESIGNADO_RELATOR_COMISSAO

    # retirado de pauta em comissão
    if row['cod_tipo_tramitacao'] == '250' and row['tipo_orgao'].startswith('Comissão'):
        return Event.RETIRADO_PAUTA_COMISSAO

    if row['cod_tipo_tramitacao'] == '250':
        return Event.RETIRADO_PAUTA_PLENARIO

    # aprovada urgência
    if row['cod_tipo_tramitacao'] == '196':
        return Event.APROVADA_URGENCIA

    # designado relator em plenário
    if row['cod_tipo_tramitacao'] == '320' and row['sigla_orgao'] == 'PLEN':
        return Event.DESIGNADO_RELATOR_PLENARIO

    # remetido
    if row['cod_tipo_tramitacao'] == '128':
        return Event.REMETIDO_AO_SENADO
    if row['cod_tipo_tramitacao'] == '609':
        return Event.REMETIDO_A_SANCAO
    if row['cod_tipo_tramitacao'] == '608':
        return Event.REMETIDO_A_PROMULGACAO
    if row['cod_tipo_tramitacao'] == '100' and row['despacho'].startswith('Remessa ao Senado Federal'):
        return Event.REMETIDO_AO_SENADO
    if row['cod_tipo_tramitacao'] == '1243' and row['despacho'].startswith('A matéria vai à sanção'):
        return Event.REMETIDO_A_SANCAO

    # aprovado em plenário
    if row['cod_tipo_tramitacao'] == '1235':
        return Event.APROVADO_PLENARIO

    # rejeitado em plenário
    if row['cod_tipo_tramitacao'] == '1236':
        return Event.REJEITADO_PLENARIO

    # arquivado
    if row['cod_tipo_tramitacao'] in ['502', '1024']:
        return Event.ARQUIVADO

    # desarquivado
    if row['cod_tipo_tramitacao'] == '503':
        return Event.DESARQUIVADO

Tomamos a tabela de tramitações, aplicamos a função acima para criar a coluna `event`, e excluímos as fileiras em que não foi atribuído valor para essa coluna. Agora temos uma tabela próxima da estrutura que queremos:

In [15]:
df_with_events = house_tram_expanded_filtered_df.copy()
df_with_events['event'] = df_with_events.apply(house_row_to_event, axis=1)
df_with_events = df_with_events[df_with_events['event'].notnull()]
df_with_events

,sequencia,data_hora,cod_tipo_tramitacao,descricao_tramitacao,id_proposicao,prop_label,cod_situacao,descricao_situacao,sigla_orgao,nome_orgao,cod_tipo_orgao,tipo_orgao,uri_orgao,despacho,regime,apreciacao,event
id_tramitacao,,,,,,,,,,,,,,,,,
771001,1,2004-05-13 14:41:00,100,Apresentação de Proposição,253500,PL 2597/2024,None,None,PLEN,Plenário,26,Plenário Virtual,https://dadosabertos.camara.leg.br/api/v2/orga...,Apresentação do Projeto de Lei pelo Dep. José ...,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,Event.APRESENTADO
770997,18,2004-06-02 09:45:00,500,Recebimento,253500,PL 2597/2024,None,None,CDE,Comissão de Desenvolvimento Econômico,2,Comissão Permanente,https://dadosabertos.camara.leg.br/api/v2/orga...,Recebimento pela CDEIC.,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,Event.RECEBIDO_COMISSAO
770996,19,2004-06-08 11:34:00,320,Designação de Relator(a),253500,PL 2597/2024,None,None,CDE,Comissão de Desenvolvimento Econômico,2,Comissão Permanente,https://dadosabertos.camara.leg.br/api/v2/orga...,"Designado Relator, Dep. Ronaldo Dimas (PSDB-TO)","Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,Event.DESIGNADO_RELATOR_COMISSAO
770979,43,2007-01-31 00:00:00,502,Arquivamento,253500,PL 2597/2024,1140,Transformado em Norma Jurídica,MESA,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora,https://dadosabertos.camara.leg.br/api/v2/orga...,Arquivado nos termos do Artigo 105 do Regiment...,"Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,Event.ARQUIVADO
770976,47,2007-06-14 10:48:00,320,Designação de Relator(a),253500,PL 2597/2024,None,None,CDE,Comissão de Desenvolvimento Econômico,2,Comissão Permanente,https://dadosabertos.camara.leg.br/api/v2/orga...,"Designado Relator, Dep. Leandro Sampaio (PPS-RJ)","Urgência (Art. 155, RICD)",Proposição Sujeita à Apreciação do Plenário,Event.DESIGNADO_RELATOR_COMISSAO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756876,14,2025-09-03 12:18:00,500,Recebimento,2529128,PL 3114/2025,925,Tramitando em Conjunto,CSAUDE,Comissão de Saúde,2,Comissão Permanente,https://dadosabertos.camara.leg.br/api/v2/orga...,Recebimento pelo(a) CSAUDE.,"Prioridade (Art. 151, II, RICD)",Proposição Sujeita à Apreciação do Plenário,Event.RECEBIDO_COMISSAO
875891,2,2025-06-30 15:09:00,100,Apresentação de Proposição,2529248,PL 3127/2025,None,None,MESA,Mesa Diretora da Câmara dos Deputados,1,Comissão Diretora,https://dadosabertos.camara.leg.br/api/v2/orga...,Apresentação do PL n. 3127/2025 (Projeto de Le...,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,Event.APRESENTADO
874103,14,2025-09-03 15:47:00,500,Recebimento,2529248,PL 3127/2025,925,Tramitando em Conjunto,CMULHER,Comissão de Defesa dos Direitos da Mulher,2,Comissão Permanente,https://dadosabertos.camara.leg.br/api/v2/orga...,Recebimento pela CMULHER.,"Ordinário (Art. 151, III, RICD)",Proposição Sujeita à Apreciação Conclusiva pel...,Event.RECEBIDO_COMISSAO


Disso podemos descartar as colunas que não desejamos e renomear desta forma:

* `id_proposicao`: Como antes;
* `prop_label`: Como antes;
* `event_ts`: O timestamp do evento;
* `event`: O valor atribuído de `Event`;
* `event_loc`: A sigla do local do evento (comissões, plenário) conforme nos dados originais.

In [16]:
house_event_df = (df_with_events[['id_proposicao', 'prop_label', 'data_hora', 'event', 'sigla_orgao']]
    .sort_values(['data_hora', 'id_proposicao'])
    .reset_index(drop=True)
    .rename(columns={
        'data_hora': 'event_ts',
        'sigla_orgao': 'event_loc',
    }))
house_event_df


,id_proposicao,prop_label,event_ts,event,event_loc
0,253500,PL 2597/2024,2004-05-13 14:41:00,Event.APRESENTADO,PLEN
1,253500,PL 2597/2024,2004-06-02 09:45:00,Event.RECEBIDO_COMISSAO,CDE
2,253500,PL 2597/2024,2004-06-08 11:34:00,Event.DESIGNADO_RELATOR_COMISSAO,CDE
3,257161,PL 2159/2021,2004-06-22 14:57:00,Event.RECEBIDO_COMISSAO,CMADS
4,259111,PL 2105/2019,2004-06-23 18:06:00,Event.APRESENTADO,PLEN
...,...,...,...,...,...
114993,2322659,PL 1203/2022,2025-09-24 15:47:00,Event.DESIGNADO_RELATOR_COMISSAO,CASP
114994,2429969,PL 1425/2024,2025-09-24 15:47:00,Event.DESIGNADO_RELATOR_COMISSAO,CASP
114995,2320831,PL 1078/2022,2025-09-24 16:30:00,Event.DESIGNADO_RELATOR_COMISSAO,CE
114996,2391466,PL 4719/2023,2025-09-24 16:52:00,Event.DESIGNADO_RELATOR_COMISSAO,CFT


### 2.2.2. Senado Federal

Vamos agora gerar uma tabela de eventos para o Senado, de estrutura semelhante àquela da Câmara, para ser concatenada a ela, gerando assim uma tabela unificada de eventos para as proposições.

A única diferença da tabela do Senado para a tabela da Câmara é que, em vez da colunas `id_proposicao` e `prop_label`, ela terá as colunas `id_processo` e `proc_label`, seguindo a nomenclatura do Senado. Isso será usado para fundir as duas tabelas depois.

No caso do Senado, a tarefa de extrair os eventos é um pouco mais complexa, porque o Senado não tem uma tabela unificada de tramitações como a Câmara (ao menos não disponível ao público). Na verdade, dentro de cada registro de processo, há uma série de payloads aninhados que trazem informações sobre os andamentos, despachos etc. Outras tramitações são encontradas nos informes legislativos e outras tabelas. Temos então que buscar os eventos um por um.

A apresentação de um processo no Senado consta na data de apresentação do documento:

In [17]:
senate_apres_df = senate_procs_df[
    senate_procs_df['documento_data_apresentacao'].notna()
].reset_index()[[
    'id_processo',
    'identificacao',
    'documento_data_apresentacao',
]].rename(
    columns={
        'identificacao': 'proc_label',
        'documento_data_apresentacao': 'event_ts',
    }
)
senate_apres_df['event'] = Event.APRESENTADO
senate_apres_df['event_loc'] = "PLEN"
senate_apres_df

,id_processo,proc_label,event_ts,event,event_loc
0,7711601,PL 547/2019,2019-01-04,Event.APRESENTADO,PLEN
1,7711690,PEC 2/2019,2019-02-06,Event.APRESENTADO,PLEN
2,7712043,PL 557/2019,2019-02-06,Event.APRESENTADO,PLEN
3,7714029,PL 577/2019,2019-02-05,Event.APRESENTADO,PLEN
4,7714041,PL 581/2019,2019-02-05,Event.APRESENTADO,PLEN
...,...,...,...,...,...
2782,8776824,PL 3337/2024,2024-12-17,Event.APRESENTADO,PLEN
2783,8779434,PL 3817/2024,2024-12-18,Event.APRESENTADO,PLEN
2784,8779801,PLP 210/2024,2024-12-18,Event.APRESENTADO,PLEN
2785,8781192,PL 4614/2024,2024-12-19,Event.APRESENTADO,PLEN


Extraímos dos Informes Legislativos as designações de relator em comissão.

In [18]:
senate_desig_df = senate_inf_df[
    senate_inf_df['descricao'].str.match(r"^(re)?distribuído", case=False)
    & senate_inf_df['id_processo'].isin(senate_procs_df.index)
].reset_index()[['id_processo', 'data_informe', 'colegiado_sigla']].join(
    senate_procs_df[['identificacao']], on="id_processo"
).rename(
    columns={
        'identificacao': 'proc_label',
        'data_informe': 'event_ts',
        'colegiado_sigla': 'event_loc',
    }
)
senate_desig_df['event'] = Event.DESIGNADO_RELATOR_COMISSAO
senate_desig_df

,id_processo,event_ts,event_loc,proc_label,event
0,7714045,2019-02-27,CAE,PLP 16/2019,Event.DESIGNADO_RELATOR_COMISSAO
1,7718136,2021-09-22,CAE,PLP 26/2019,Event.DESIGNADO_RELATOR_COMISSAO
2,7720504,2019-03-22,CDH,PL 938/2019,Event.DESIGNADO_RELATOR_COMISSAO
3,7723055,2021-10-15,CE,PL 1265/2019,Event.DESIGNADO_RELATOR_COMISSAO
4,7714051,2019-03-14,CAE,PL 579/2019,Event.DESIGNADO_RELATOR_COMISSAO
...,...,...,...,...,...
1854,8737346,2024-12-02,CMO,PLN 38/2024,Event.DESIGNADO_RELATOR_COMISSAO
1855,8762115,2024-12-06,CMO,PLN 39/2024,Event.DESIGNADO_RELATOR_COMISSAO
1856,8682454,2024-08-06,CMO,PLN 16/2024,Event.DESIGNADO_RELATOR_COMISSAO
1857,8726711,2024-12-02,CMO,PLN 28/2024,Event.DESIGNADO_RELATOR_COMISSAO


E das unidades de destino, a recepção em Comissão:

In [19]:
senate_rec_com_df = (
    senate_unid_df.join(senate_desp_df, on="id_despacho", lsuffix="_unid", rsuffix="_desp")
        .join(senate_prov_df, on="id_providencia", lsuffix="_unid", rsuffix="_prov")
        .join(senate_procs_df[['identificacao']], on="id_processo")
        .reset_index()
)[[
    'id_processo_unid',
    'identificacao',
    'data_despacho',
    'colegiado_sigla',
]].rename(
    columns={
        'id_processo_unid': 'id_processo',
    'identificacao': 'proc_label',
    'data_despacho': 'event_ts',
    'colegiado_sigla': 'event_loc',
    }
)
senate_rec_com_df['event'] = Event.RECEBIDO_COMISSAO
senate_rec_com_df

,id_processo,proc_label,event_ts,event_loc,event
0,7711601,PL 547/2019,2019-02-07,CAE,Event.RECEBIDO_COMISSAO
1,7711690,PEC 2/2019,2019-02-07,CCJ,Event.RECEBIDO_COMISSAO
2,7712043,PL 557/2019,2019-02-07,CRE,Event.RECEBIDO_COMISSAO
3,7714029,PL 577/2019,2019-02-11,CAE,Event.RECEBIDO_COMISSAO
4,7714041,PL 581/2019,2022-12-15,PLEN,Event.RECEBIDO_COMISSAO
...,...,...,...,...,...
130088,8555503,PL 4764/2023,2023-10-17,CCDD,Event.RECEBIDO_COMISSAO
130089,8565029,PL 2245/2023,2023-11-14,CAE,Event.RECEBIDO_COMISSAO
130090,8575549,PL 5289/2023,2023-11-13,CDR,Event.RECEBIDO_COMISSAO
130091,8511486,PL 3/2023,2023-08-17,CDH,Event.RECEBIDO_COMISSAO


Da mesma forma, extraímos as aprovações de urgência, retiradas de pauta, designações de relator em plenário. Isso obtemos com busca por palavra-chave no corpo do informe legislativo.

In [20]:

desig_relat_re = re.compile(r'(?i)^designad.*relator')
senate_desig_plen_df = senate_inf_df[
    senate_inf_df['colegiado_sigla'].eq("PLEN")
    & senate_inf_df['descricao'].str.match(desig_relat_re)
].reset_index()[[
    'id_processo',
    'data_informe',
    'colegiado_sigla',
]].join(senate_procs_df[['identificacao']], on='id_processo').rename(
    columns={
        'data_informe': 'event_ts',
        'colegiado_sigla': 'event_loc',
        'identificacao': 'proc_label',
    }
)
senate_desig_plen_df['event'] = Event.DESIGNADO_RELATOR_PLENARIO
senate_desig_plen_df

,id_processo,event_ts,event_loc,proc_label,event
0,7872006,2022-12-21,PLEN,PL 509/2020,Event.DESIGNADO_RELATOR_PLENARIO
1,7749718,2021-03-11,PLEN,PL 2706/2019,Event.DESIGNADO_RELATOR_PLENARIO
2,7778390,2023-05-02,PLEN,PL 3983/2019,Event.DESIGNADO_RELATOR_PLENARIO
3,7815156,2022-05-02,PLEN,PL 5189/2019,Event.DESIGNADO_RELATOR_PLENARIO
4,7833514,2023-05-10,PLEN,PLP 245/2019,Event.DESIGNADO_RELATOR_PLENARIO
...,...,...,...,...,...
292,8779801,2024-12-19,PLEN,PLP 210/2024,Event.DESIGNADO_RELATOR_PLENARIO
293,8670267,2024-05-27,PLEN,PL 1213/2024,Event.DESIGNADO_RELATOR_PLENARIO
294,8663585,2024-07-09,PLEN,PL 1847/2024,Event.DESIGNADO_RELATOR_PLENARIO
295,8746458,2024-11-12,PLEN,PLP 175/2024,Event.DESIGNADO_RELATOR_PLENARIO


In [21]:

aprov_urgencia_re = re.compile(r'(?i)aprovado[^\n]*urgência')
senate_aprov_urg_df = senate_inf_df[
    senate_inf_df['descricao'].str.contains(aprov_urgencia_re)
].reset_index()[[
    'id_processo',
    'data_informe',
    'colegiado_sigla',
]].join(senate_procs_df[['identificacao']], on='id_processo').rename(
    columns={
        'data_informe': 'event_ts',
        'colegiado_sigla': 'event_loc',
        'identificacao': 'proc_label',
    }
)
senate_aprov_urg_df['event'] = Event.APROVADA_URGENCIA
senate_aprov_urg_df

,id_processo,event_ts,event_loc,proc_label,event
0,7718337,2023-05-30,PLEN,PL 776/2019,Event.APROVADA_URGENCIA
1,7720308,2023-07-05,PLEN,PLP 41/2019,Event.APROVADA_URGENCIA
2,7716769,2020-02-18,PLEN,PLP 19/2019,Event.APROVADA_URGENCIA
3,7837621,2023-08-29,PLEN,PLP 257/2019,Event.APROVADA_URGENCIA
4,7859496,2020-05-25,None,MPV 918/2020,Event.APROVADA_URGENCIA
...,...,...,...,...,...
337,8718106,2024-09-17,PLEN,PL 1725/2024,Event.APROVADA_URGENCIA
338,8758266,2024-12-11,CMA,PL 3944/2024,Event.APROVADA_URGENCIA
339,8672212,2024-06-04,PLEN,PL 914/2024,Event.APROVADA_URGENCIA
340,8542290,2023-11-29,PLEN,PL 3626/2023,Event.APROVADA_URGENCIA


In [22]:

retir_pauta_re = re.compile(r'(?i)a matéria (foi|é) retirada de pauta')
senate_retir_paut_df = senate_inf_df[
    senate_inf_df['colegiado_sigla'].ne("PLEN")
    & senate_inf_df['descricao'].str.contains(retir_pauta_re)
].reset_index()[[
    'id_processo',
    'data_informe',
    'colegiado_sigla',
]].join(senate_procs_df[['identificacao']], on='id_processo').rename(
    columns={
        'data_informe': 'event_ts',
        'colegiado_sigla': 'event_loc',
        'identificacao': 'proc_label',
    }
)
senate_retir_paut_df['event'] = Event.RETIRADO_PAUTA_COMISSAO
senate_retir_paut_df

/tmp/ipykernel_3264518/1411710615.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  & senate_inf_df['descricao'].str.contains(retir_pauta_re)


,id_processo,event_ts,event_loc,proc_label,event
0,7718146,2020-02-12,CCJ,PL 711/2019,Event.RETIRADO_PAUTA_COMISSAO
1,7714289,2019-05-21,CAE,PL 583/2019,Event.RETIRADO_PAUTA_COMISSAO
2,7711690,2019-05-08,CCJ,PEC 2/2019,Event.RETIRADO_PAUTA_COMISSAO
3,7714029,2019-04-17,CAS,PL 577/2019,Event.RETIRADO_PAUTA_COMISSAO
4,7714289,2019-05-21,CAE,PL 583/2019,Event.RETIRADO_PAUTA_COMISSAO
...,...,...,...,...,...
219,8658538,2024-09-17,CE,PL 1711/2024,Event.RETIRADO_PAUTA_COMISSAO
220,8658538,2024-09-03,CE,PL 1711/2024,Event.RETIRADO_PAUTA_COMISSAO
221,8488172,2023-09-27,CDH,PL 3324/2023,Event.RETIRADO_PAUTA_COMISSAO
222,8574430,2024-03-13,CAS,PL 3466/2023,Event.RETIRADO_PAUTA_COMISSAO


E finalmente as situações iniciadas conforme indicado nos informes legislativos

In [23]:

def map_sig(sig):
    if pd.isna(sig):
        return sig
        
    if sig.startswith("APRVD"):
        return Event.APROVADO_PLENARIO
    if sig == "ARQVD":
        return Event.ARQUIVADO
    if sig == "RTPA":
        return Event.RETIRADO_PAUTA_PLENARIO
    if sig == "RMSAN":
        return Event.REMETIDO_A_SANCAO
    if sig == "RMCD":
        return Event.REMETIDO_A_CAMARA
    if sig == "RMPRO":
        return Event.REMETIDO_A_PROMULGACAO
    if sig == "DESARQLEG":
        return Event.DESARQUIVADO
    if sig.startswith("RJTDA"):
        return Event.REJEITADO_PLENARIO

senate_other_df = senate_inf_df.join(
    senate_inf_df['sigla_situacao_iniciada'].apply(map_sig).rename('event')  # type: ignore
)

senate_other_df = senate_other_df[senate_other_df['event'].notna()][[
    'id_processo',
    'data_informe',
    'event'
]].join(
    senate_procs_df[['identificacao']],
    on='id_processo'
).rename(columns={
    'data_informe': 'event_ts',
    'identificacao': 'proc_label',
})
# Na tabela original pode ser Plenário, Mesa ou vazio, mas tudo isso é relativo ao plenário
senate_other_df['event_loc'] = "PLEN"
senate_other_df

,id_processo,event_ts,event,proc_label,event_loc
id_informe_legislativo,,,,,
1965,7712043,2022-12-16,Event.REMETIDO_A_CAMARA,PL 557/2019,PLEN
1967,7714041,2022-12-20,Event.REMETIDO_A_CAMARA,PL 581/2019,PLEN
1970,7714055,2023-10-04,Event.REMETIDO_A_CAMARA,PL 580/2019,PLEN
1979,7717212,2022-12-22,Event.ARQUIVADO,PL 634/2019,PLEN
1984,7717482,2023-04-13,Event.REMETIDO_A_CAMARA,PL 633/2019,PLEN
...,...,...,...,...,...
315055,8708174,2024-08-29,Event.APROVADO_PLENARIO,PLN 25/2024,PLEN
315106,8726711,2024-12-19,Event.REMETIDO_A_SANCAO,PLN 28/2024,PLEN
315571,8671111,2024-05-29,Event.APROVADO_PLENARIO,PLN 13/2024,PLEN


Construímos a tabela de eventos do Senado:

In [24]:
senate_event_df = pd.concat([
    senate_apres_df,
    senate_desig_df,
    senate_rec_com_df,
    senate_desig_plen_df,
    senate_aprov_urg_df,
    senate_retir_paut_df,
    senate_other_df,
])
senate_event_df

,id_processo,proc_label,event_ts,event,event_loc
0,7711601,PL 547/2019,2019-01-04,Event.APRESENTADO,PLEN
1,7711690,PEC 2/2019,2019-02-06,Event.APRESENTADO,PLEN
2,7712043,PL 557/2019,2019-02-06,Event.APRESENTADO,PLEN
3,7714029,PL 577/2019,2019-02-05,Event.APRESENTADO,PLEN
4,7714041,PL 581/2019,2019-02-05,Event.APRESENTADO,PLEN
...,...,...,...,...,...
315055,8708174,PLN 25/2024,2024-08-29,Event.APROVADO_PLENARIO,PLEN
315106,8726711,PLN 28/2024,2024-12-19,Event.REMETIDO_A_SANCAO,PLEN
315571,8671111,PLN 13/2024,2024-05-29,Event.APROVADO_PLENARIO,PLEN
315688,8726711,PLN 28/2024,2024-12-18,Event.APROVADO_PLENARIO,PLEN


### 2.2.3. Tabela conjunta

Agora criamos a tabela conjunta. Primeiros rotulamos cada fileira com a tabela original e concatenamos.

As colunas `id_proposicao` e `id_processo` são temporariamente descartadas, e unificamos as colunas `*_label` para que sirvam como referencial comum.

In [25]:
senate_event_df['casa'] = "senado"
house_event_df['casa'] = "camara"

In [26]:
full_event_df = pd.concat([
    house_event_df.drop('id_proposicao', axis=1).rename(columns={'prop_label': 'label'}),
    senate_event_df.drop('id_processo', axis=1).rename(columns={'proc_label': 'label'})
], ignore_index=True).sort_values(['label', 'event_ts']).reset_index(drop=True)
full_event_df

,label,event_ts,event,event_loc,casa
0,MPV 1000/2020,2020-09-03 00:00:00,Event.APRESENTADO,PLEN,senado
1,MPV 1000/2020,2020-09-03 10:57:00,Event.APRESENTADO,EXEC,camara
2,MPV 1000/2020,2020-09-09 00:00:00,Event.RECEBIDO_COMISSAO,MESA,camara
3,MPV 1000/2020,2020-09-09 00:00:00,Event.APRESENTADO,MESA,camara
4,MPV 1000/2020,2020-12-17 00:00:00,Event.DESIGNADO_RELATOR_PLENARIO,PLEN,camara
...,...,...,...,...,...
253874,PLV 8/2023,2023-03-30 17:23:00,Event.APRESENTADO,MPV115222,camara
253875,PLV 9/2020,2020-04-29 22:15:00,Event.APRESENTADO,PLEN,camara
253876,PLV 9/2021,2021-05-25 19:23:00,Event.APRESENTADO,PLEN,camara
253877,PLV 9/2022,2022-05-11 17:23:00,Event.APRESENTADO,MPV108021,camara


Vamos remover repetições _consecutivas_ do mesmo evento para a mesma proposição no mesmo lugar.

In [27]:
full_event_df = full_event_df[full_event_df[['label', 'event', 'casa']].ne(
    full_event_df[['label', 'event', 'casa']].shift()
).any(axis=1)].reset_index(drop=True)
full_event_df

,label,event_ts,event,event_loc,casa
0,MPV 1000/2020,2020-09-03 00:00:00,Event.APRESENTADO,PLEN,senado
1,MPV 1000/2020,2020-09-03 10:57:00,Event.APRESENTADO,EXEC,camara
2,MPV 1000/2020,2020-09-09 00:00:00,Event.RECEBIDO_COMISSAO,MESA,camara
3,MPV 1000/2020,2020-09-09 00:00:00,Event.APRESENTADO,MESA,camara
4,MPV 1000/2020,2020-12-17 00:00:00,Event.DESIGNADO_RELATOR_PLENARIO,PLEN,camara
...,...,...,...,...,...
99990,PLV 8/2023,2023-03-30 17:23:00,Event.APRESENTADO,MPV115222,camara
99991,PLV 9/2020,2020-04-29 22:15:00,Event.APRESENTADO,PLEN,camara
99992,PLV 9/2021,2021-05-25 19:23:00,Event.APRESENTADO,PLEN,camara
99993,PLV 9/2022,2022-05-11 17:23:00,Event.APRESENTADO,MPV108021,camara


Finalmente, atribuímos as tags dos vértices - que equivalem aos valores desprezados de `id_proposicao` e `id_processo`. Para qualquer proposição que tenha tramitado tanto na Câmara quanto no Senado, teremos ambas as tags naquela fileira de vento.

In [28]:
full_event_df = full_event_df.join(
    house_props_df.set_index('prop_label')['prop_tag'],
    on="label"
).join(
    senate_procs_df.set_index('identificacao')[['tag']].rename(columns={'tag': 'proc_tag'}),
    on="label",
)

In [29]:
full_event_df


,label,event_ts,event,event_loc,casa,prop_tag,proc_tag
0,MPV 1000/2020,2020-09-03 00:00:00,Event.APRESENTADO,PLEN,senado,CP:2262062,SP:7979012
1,MPV 1000/2020,2020-09-03 10:57:00,Event.APRESENTADO,EXEC,camara,CP:2262062,SP:7979012
2,MPV 1000/2020,2020-09-09 00:00:00,Event.RECEBIDO_COMISSAO,MESA,camara,CP:2262062,SP:7979012
3,MPV 1000/2020,2020-09-09 00:00:00,Event.APRESENTADO,MESA,camara,CP:2262062,SP:7979012
4,MPV 1000/2020,2020-12-17 00:00:00,Event.DESIGNADO_RELATOR_PLENARIO,PLEN,camara,CP:2262062,SP:7979012
...,...,...,...,...,...,...,...
99990,PLV 8/2023,2023-03-30 17:23:00,Event.APRESENTADO,MPV115222,camara,CP:2354532,NaN
99991,PLV 9/2020,2020-04-29 22:15:00,Event.APRESENTADO,PLEN,camara,CP:2250966,NaN
99992,PLV 9/2021,2021-05-25 19:23:00,Event.APRESENTADO,PLEN,camara,CP:2284649,NaN
99993,PLV 9/2022,2022-05-11 17:23:00,Event.APRESENTADO,MPV108021,camara,CP:2322707,NaN


Gravamos então para uso nas etapas seguintes.

In [30]:
full_event_df.to_pickle(ACCESS_DIR / 'full_event_df.pkl')

Fazemos uma breve inspeção.

In [31]:
event_summary_df = full_event_df.value_counts(['event', 'casa']).unstack()
event_summary_df


casa,camara,senado
event,,
Event.APRESENTADO,28958.0,2789.0
Event.RECEBIDO_COMISSAO,31097.0,2313.0
Event.DESIGNADO_RELATOR_COMISSAO,16615.0,1335.0
Event.RETIRADO_PAUTA_COMISSAO,3904.0,165.0
Event.RETIRADO_PAUTA_PLENARIO,NaN,192.0
Event.APROVADA_URGENCIA,1425.0,333.0
Event.DESIGNADO_RELATOR_PLENARIO,1610.0,294.0
Event.REMETIDO_AO_SENADO,599.0,NaN
Event.REMETIDO_A_CAMARA,NaN,741.0


In [32]:
full_event_df.groupby('casa').count()

,label,event_ts,event,event_loc,prop_tag,proc_tag
casa,,,,,,
camara,89530,89530,89530,89530,89530,12210
senado,10508,10508,10508,10402,7621,10508
